In [2]:
from searchtweets import load_credentials, collect_results, gen_request_parameters
import yaml
import pandas as pd
import pickle
import os
import sys
pd.set_option('display.max_colwidth', -1)

<ipython-input-2-560ffd9804b2>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [5]:
API_KEY = 'adCrfdHEX7bw6xYJdqeWrdMJJ'
API_SECRET_KEY = 'La2QnkdEVSySW4lWHyy66KfAqIHAiRdUTkPEabUBHujKRFGf2I'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAACmIOAEAAAAAO9z8EgwBJp95TWjWwdrAItcL6GM%3DnZCxxZk1qA3gUKXaKJI2ITTNNNbcIWyY2NcnHbkwplUIvFe3Q0'
RESULTS_PER_CALL = 100 #1번에 10000개의 트윗

In [6]:
## Search keywords
# 단순 키워드로 트윗을 수집할 경우 별도의 설정 없이 키워드를 직접 입력
# @ = matching tweet with the given username / # = matching tweet with the given hashtag
# from: X = specific username x / -is:reply (or retweet) = exclude reply (retweet) / 유저네임은 반드시 트위터 프로필에서 @ 다음에 있는 명칭으로 사용할 것
# query 관련해서는 https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query 참고

# SEARCH_QUERY = 'from:United_Purpose' # from: X = specific username x / -is:reply (or retweet) = exclude reply (retweet)
SEARCH_QUERY = 'human rights -is:retweet' # 현재 twitter api v2에서는 리트윗의 경우 트윗 전문을 가져오지 못하는 이슈가 있어서 본 내용에서는 제외하고 수집
# START_TIME = '2021-03-26'  # format YYYY-MM-DD / 현재 시점에서 일주일 내의 자료만 확보할 수 있기 때문에 일주일 내의 특정 시점에서만 자료를 수집하고 싶을 경우
# END_TIME = '2021-03-23' # format YYYY-MM-DD
MAX_TWEETS = 1000 # 특정 유저가 아니라 특정 키워드로 추출하여 모으는 트윗 개수를 제한할 필요가 있을 경우

In [11]:
config = dict(
    search_tweets_v2=dict(
        endpoint="https://api.twitter.com/2/tweets/search/recent", #정해진거임
        consumer_key=API_KEY,
        consumer_secret=API_SECRET_KEY,
        bearer_token=BEARER_TOKEN
    )
)

with open('twitter_keys.yaml', 'w') as config_file: # Save twitter API code into local computer as yaml format
    yaml.dump(config, config_file, default_flow_style=False) # yaml 형태로 저장할 때 yaml 자체의 자료저장 형태가 아닌 config에서 지정한 dictionary 형태로 저장하도록 default_flow_style=False 로 해줌

In [12]:
## Load twitter API code from previous setting

search_args = load_credentials("twitter_keys.yaml", # load_credentials: yaml 파일에서 config에서 search_tweets_v2 로 지정해놓은 변수를 찾으려고 시도
                               yaml_key = "search_tweets_v2"
                               )

In [14]:
## Set parameters (https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet 참고)
# tweet_fields에서 가져오고 싶은 메타데이터를 획득가능, 단 tweet_fields="" 에 들어가는 항목들은 반드시 띄어쓰기 없이 comma 단위로 나열할 것

rule = gen_request_parameters(query = SEARCH_QUERY,
                              results_per_call = RESULTS_PER_CALL,
                              tweet_fields = "public_metrics,created_at,entities" # public_metrics: retweet+quote+favorite number / created_at: time tweet created / entities: url etc.
                              )

In [15]:
## Collect data
# request time limit 를 넘어버릴 경우 HTTP Error code: 429: Rate limit exceeded 가 뜰 수 있음 450 request/15 min (실제 자료 수집엔 영향 없음)

TWEETS = collect_results(rule,
                         max_tweets=MAX_TWEETS,
                         result_stream_args=search_args)

len(TWEETS) # 위에선 MAX_TWEETS = 1000 으로 했는데 왜 1010개가 수집되었는지 아래에서 설명

1010

In [17]:
num = TWEETS[0]['id']
print(f'https://twitter.com/twitter/status/{num}') # tweet url

print(TWEETS[0]['created_at'][0:10]) # the date tweet written
print(TWEETS[0]['text'].replace('\n', ' ')) # tweet text

https://twitter.com/twitter/status/1376831743764832258
2021-03-30
As Bangladesh celebrated independence, human rights groups criticised the government for what they described as growing authoritarianism, including forced disappearances and extrajudicial killings. #MythsVsRealityBangladesh https://t.co/tYcRzdePIO


In [18]:
with open('TWEETS_API.pickle', 'wb') as f:
    pickle.dump(TWEETS, f)

In [19]:
import pprint

pprint.pprint(TWEETS[0])

{'created_at': '2021-03-30T09:40:42.000Z',
 'entities': {'annotations': [{'end': 12,
                               'normalized_text': 'Bangladesh',
                               'probability': 0.9766,
                               'start': 3,
                               'type': 'Place'}],
              'hashtags': [{'end': 223,
                            'start': 198,
                            'tag': 'MythsVsRealityBangladesh'}],
              'urls': [{'display_url': 'pic.twitter.com/tYcRzdePIO',
                        'end': 247,
                        'expanded_url': 'https://twitter.com/T21303/status/1376831743764832258/photo/1',
                        'start': 224,
                        'url': 'https://t.co/tYcRzdePIO'},
                       {'display_url': 'pic.twitter.com/tYcRzdePIO',
                        'end': 247,
                        'expanded_url': 'https://twitter.com/T21303/status/1376831743764832258/photo/1',
                        'start': 224,
  

In [20]:
DATE = []
RETWEET = []
QUOTE = []
LIKE = []
URL = []
TEXT = []

## Text with metadata from pre-collected tweets

for tweet in TWEETS:

    try:
        DATE.append(tweet['created_at'][0:10])
        
        RETWEET.append(tweet['public_metrics']['retweet_count'])
        
        QUOTE.append(tweet['public_metrics']['quote_count'])
                
        LIKE.append(tweet['public_metrics']['like_count'])
        
        num = tweet['id']
        URL.append(f'https://twitter.com/twitter/status/{num}')
        
        TEXT.append(tweet['text'].replace('\n', ' '))
        
    except KeyError: # 트위터 본문이 없거나 api를 통해서 수집되지 않는 메타데이터가 발생했을 경우 'None' 텍스트를 대신 넣고 passing
        DATE.append('NONE')
        RETWEET.append('NONE')
        QUOTE.append('NONE')
        LIKE.append('NONE')
        URL.append('NONE')
        TEXT.append('NONE')

RT = RETWEET + QUOTE

## Save as dataframe format
DF = pd.DataFrame(list(zip(DATE, LIKE, RT, URL, TEXT)), columns =['Date', 'Likes', 'Retweet', 'Url', 'Text'])
DF.head()

,Date,Likes,Retweet,Url,Text
0,2021-03-30,0,0,https://twitter.com/twitter/status/1376831743764832258,"As Bangladesh celebrated independence, human rights groups criticised the government for what they described as growing authoritarianism, including forced disappearances and extrajudicial killings. #MythsVsRealityBangladesh https://t.co/tYcRzdePIO"
1,2021-03-30,0,0,https://twitter.com/twitter/status/1376831720180355076,Lots of to be optimistic about @DubravkaSRVAW ✔️More human rights instruments in place ✔️Greater reporting on VAWG ✔️Women speaking out and claiming their rights BUT ❌COVID-19 exacerbating violence/limiting rights ❌Human rights mechanisms not ratified by all
2,2021-03-30,0,0,https://twitter.com/twitter/status/1376831708872515586,@originals_1878 @LeeEdwardLamon1 you've got a point but supporting the dictatorship in a country which is opposed to human rights isn't the logical way to go about it.
3,2021-03-30,0,0,https://twitter.com/twitter/status/1376831702404714498,Kadiri si Roque amputa. Sure kayong human rights lawyer yan before? Myghad pisting yawa
4,2021-03-30,0,0,https://twitter.com/twitter/status/1376831688135876608,This is a human rights crime! https://t.co/H10Dqv0uZt


In [21]:
idx = DF[(DF['Date'] == 'NONE') & (DF['Url'] == 'NONE') & (DF['Text'] == 'NONE')].index
DF1 = DF.drop(idx)
print(DF1.shape[0])

1000


In [22]:
## Save as dataframe format
DF1.to_csv('TWEETS.csv', encoding = 'utf-8-sig', index_label = False, index = False)